# Language Localization on Live Audio


Importing Localization File

In [ ]:
from languageLocalization import *

In [ ]:
# Loading Model
model = initModel('weights.hdf5')
model.compile(loss = 'categorical_crossentropy', optimizer = 'adamax', metrics = ['accuracy'])

Some Hypterparameters

In [ ]:
chunk_duration = 1 # Each read length in seconds from mic.
fs = 16000 # sampling rate for mic
chunk_samples = int(fs * chunk_duration) # Each read length in number of samples.

# Each model input data duration in seconds, need to be an integer numbers of chunk_duration
feed_duration = 1
feed_samples = int(fs * feed_duration)
assert feed_duration/chunk_duration == int(feed_duration/chunk_duration)

Below script will run audio localization on live audio. 
Some of the parameters to use here are:
    amnt_time -- the amount of time (in seconds) to run live recording
    silence_threshold -- amount of dB considered to be silence. 

Use a high silence threshold to make predictions between languages more easier

In [6]:
def get_audio_input_stream(callback):
  stream = pyaudio.PyAudio().open(
	  format=pyaudio.paInt16,
	  channels=1,
	  rate=fs,
	  input=True,
	  frames_per_buffer=chunk_samples,
	  input_device_index=0,
	  stream_callback=callback)
  return stream

# Starting Live Recording
# Queue to communiate between the audio callback and main thread

q = Queue()

run = True

silence_threshold = 1000
amnt_time = 30

# Run the demo for a timeout seconds
timeout = time.time() + amnt_time  # 0.5 minutes from now

# Data buffer for the input wavform
data = np.zeros(feed_samples, dtype='int16')

def callback(in_data, frame_count, time_info, status):
    global run, timeout, data, silence_threshold    
    if time.time() > timeout:
        run = False        
    data0 = np.frombuffer(in_data, dtype='int16')
    if np.abs(data0).mean() < silence_threshold:
        sys.stdout.write('-')
        return (in_data, pyaudio.paContinue)
    else:
        sys.stdout.write('.')
    data = np.append(data,data0)    
    if len(data) > feed_samples:
        data = data[-feed_samples:]
        # Process data async by sending a queue.
        q.put(data)
    return (in_data, pyaudio.paContinue)

stream = get_audio_input_stream(callback)
stream.start_stream()
try:
    while run:
        data = q.get()
        preds = predict(data, model)
        print(preds)
        
except (KeyboardInterrupt, SystemExit):
    stream.stop_stream()
    stream.close()
    timeout = time.time()
    run = False
        
stream.stop_stream()
stream.close()

....urdu
urdu
.english
.english
.urdu
.urdu
.sindhi
.sindhi
.english
.english
.urdu
.urdu
.english
.english
.english
.english
.urdu
.urdu
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.english
.urdu
...........